# Correlación entre muchos (big data) pares de series de tiempo unidimensionales


<a name="contents"></a>
# Contenido

- [Notas, desconexión y librerías](#conecct)
- [Funciones para graficar con 2 ejes 'y'](#plotFuncts)
- [Importando datos de 41 activos financieros](#data)
- [Requerimientos para usar el modelo (cargar los datos previamente)](#reqs)
- [Conclusiones](#Conclu)

Volver a [Tabla de contenidos](#contents).


<a name="conecct"></a>
# Notas, desconexión y librerías:
- La primera celda es un código que en teoría evita la desconexión de Colab.

Volver a [Tabla de contenidos](#contents).

In [ ]:
#Source: https://towardsdatascience.com/10-tips-for-a-better-google-colab-experience-33f8fe721b82#0d57
#Stop Colab from disconecting:
#ctl+shift+i -> console -> command lines:
function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking,60000)

### Importando paquetes:

In [ ]:
#To import data
#!pip install pandas_datareader #For COLAB #Use if yfinance does not work #import pandas_datareader.data as web
#installing yfinance (Colab)
!pip install yfinance
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
#Data managing
import numpy as np
import os
import pandas as pd
#Graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
#RNN
import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
from sklearn.metrics import mean_squared_error
#For Bootstrap
!pip install dc_stat_think
import dc_stat_think as dcst
#I dont know for what are these packages
import seaborn as sns
sns.set(font_scale=1.1)
import time
import gc
import sys
#To use goodness fit test
from scipy import stats
import random

#Setting figure size
mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['axes.grid'] = False

     |████████████████████████████████| 6.3 MB 41.0 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=f23cc710317497e13a2ca84e0669b88c0976c07ec07f51d16a117614cd72f471
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


<a name="plotFuncts"></a>
# Funciones para graficar con 2 ejes 'y'
- *Candidata a exportar* 

Volver a [Tabla de contenidos](#contents).

In [ ]:
#NOT USED - DELETE IT
def plot_2y_ax(x, y1, y2, title, xlab, y1_lab, y2_lab, col1, col2, mark):
  #1st plot
  fig, ax1 = plt.subplots()
  ax1.set_xlabel(xlab, size=15)
  ax1.set_ylabel(y1_lab, color = col1, size = 15)
  ax1.plot(x,y1,color=col1,marker=mark)
  ax1.tick_params(axis='y', labelcolor=col1)
  ax1.tick_params(labelsize=13)
  #2nd plot
  ax2 = ax1.twinx()
  ax2.set_ylabel(y2_lab, color = col2, size = 15)
  ax2.plot(x,y2,color=col2,marker=mark)
  ax2.tick_params(axis='y', labelcolor=col2)
  ax2.tick_params(labelsize=13)
  #Title and showing
  plt.title(title, size=19)
  plt.show()

<a name="data"></a>
#Importando datos de 41 activos financieros
- 3 criptomonedas.
- 8 accciones.
- 1 ETF.
- 10 Futuros.
- 9 índices
- 10 divisas.

Volver a [Tabla de contenidos](#contents).

In [ ]:
%%time
print("--------------------------DATA--------------------------")
from pandas.core.common import flatten
CRYPTO = ["BTC-USD","ETH-USD","USDT-USD"]
#print(len(CRYPTO))7-data2D
#STOCKS = ["AMC","AAPL","CLNE","TSLA","SYKE","ZHUZF","ANPDF","MVIS","SSL","BGCP"]#4 most actives 3 gainers 3 losers
STOCKS = ["AMC","AAPL","CLNE","TSLA","SYKE","MVIS","SSL","BGCP"]#4 most actives 1 gainers 3 losers
ETFS = ["QQQ"]
FUTS = ["ES=F","YM=F","NQ=F","ZB=F","ZN=F","ZF=F","ZT=F","GC=F","MGC=F","SI=F"]
INDICES = ["^GSPC","^DJI","^IXIC","^NYA","^XAX","^RUT","^VIX","^AXJO","^AORD"]
CURRENCIES = ["EURUSD=X","JPY=X","GBPUSD=X","AUDUSD=X","NZDUSD=X","EURJPY=X","GBPJPY=X","EURGBP=X","EURCAD=X","EURSEK=X"]
#1408(6)
#COMMODITIES = []
DATASET =[CRYPTO,ETFS,FUTS,INDICES,CURRENCIES,STOCKS]#,COMMODITIES]#1421
##print(list(flatten(DATASET)))
df_adj_lot = pdr.get_data_yahoo(list(flatten(DATASET))).dropna()["Adj Close"]
#print(type(df_adj_lot))
#print(df_adj_lot.head(4))
#print(df_adj_lot.tail(4))
print(df_adj_lot.describe())

--------------------------DATA--------------------------
[*********************100%***********************]  41 of 41 completed
              AAPL          AMC  ...         ^VIX         ^XAX
count  1420.000000  1420.000000  ...  1420.000000  1420.000000
mean     56.097574    14.891964  ...    18.044930  2429.349760
std      34.639075     8.729835  ...     8.425502   285.928704
min      21.098566     1.980000  ...     9.140000  1317.219971
25%      28.318497     9.360387  ...    12.665000  2274.347412
50%      42.760361    13.295254  ...    15.625000  2462.929932
75%      69.059765    20.487144  ...    21.242500  2586.905090
max     149.149994    62.549999  ...    82.690002  3320.620117

[8 rows x 41 columns]
CPU times: user 5.13 s, sys: 269 ms, total: 5.4 s
Wall time: 6.78 s


<a name="reqs"></a>
# Requerimientos para usar el modelo (cargar los datos previamente)

Volver a [Tabla de contenidos](#contents).

### Funciones a usar 

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  ''' 
      dataset: Array containing the whole time series (stadarized or not)
      start_index: index from where data start to analize
      end_index: final index to take data. None is to take all data
      history_size: window size to create sequence (and then used to predict)
      target_size: size of data to predict
  '''
  hist = [] #hist = history
  targets = [] 

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    #window = window of time
    window = range(i-history_size,i) #it balances the start_index = start_index + history_size made above
    #print(window,i)
    #print(window)
    #Reshape hist from (history_size,) to (history_size,1)
    hist.append(np.reshape(dataset[window], (history_size, 1))) #len(dataset[window]) = history_size
    #print(hist)
    targets.append(dataset[i+target_size-1])#note: the "-1" is not in the original code"
  return np.array(hist), np.array(targets)

def my_model(n_of_neurons, in_shape):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.LSTM(units=n_of_neurons, input_shape=in_shape))
  model.add(tf.keras.layers.Dense(1))
  return model

## Variables

In [ ]:
#-------------------- HYPERPARAMETERS ----------------------
TRAIN_SPLIT = int(len(df_adj_lot)*0.8)

#Data load params
BATCH_SIZE = 256
BUFFER_SIZE = 10000
#TRAIN_SPLIT = int(len(df_btc)*0.8) #Put before

#LSTM params
EVALUATION_INTERVAL = 200 #FOR WHAT?
EPOCHS = 100
PATIENCE = 5

#Seeding for reproducibility
SEED = 13
tf.random.set_seed(SEED)

#-------------------- FOR TRAINING AND VALIDATION DATA ----------------------
#Defining history and target size
uni_past_history = 20
uni_future_target = 1 #I DONT KNOW WHY 0 AND NOT 1

## Corriendo el modelo con cada activo financiero:
Se corre el modelo ganador (univariado de un paso con una capa intermedia de 256 neuronas) N_SIMS = 5 veces por cada uno de los N_DATA = 41 activos financieros 

In [ ]:
%%time
#print(len(df_adj_lot))
#print(len(df_adj_lot.columns))
print(df_adj_lot)
#print(df_adj_lot.iloc[:,1])
#print(type(df_adj_lot.iloc[:,1]))
#print(type(df_adj_lot.columns[0]))
#print(df_adj_lot.columns[0])

df_errors = pd.DataFrame()

#print(df_errors)
N_DATA = len(df_adj_lot.columns)
#N_DATA = 2
N_SIMS = 10#3#Number of simulations per financial assets
for j in range(N_DATA):
  print("Asset:"+str(j+1))
  errors_sim = []

  #------ GETTING DATA TO WORK (univariate time series)----
  uni_data = df_adj_lot.iloc[:,j]
  #print(uni_data)
  uni_data = uni_data.values

  #-------------------- STANDARIZATION ----------------------
  uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
  uni_train_std = uni_data[:TRAIN_SPLIT].std()
  uni_data = (uni_data - uni_train_mean)/uni_train_std

  #-------------------- TRAINING AND VALIDATION DATA ----------------------
  #Creating training data
  x_train_single_uni, y_train_single_uni = univariate_data(dataset = uni_data, start_index = 0, 
                                            end_index = TRAIN_SPLIT, history_size=uni_past_history,
                                            target_size = uni_future_target)

  #Creating validation (for prediction) data
  x_val_single_uni, y_val_single_uni = univariate_data(dataset = uni_data, start_index = TRAIN_SPLIT,
                                        end_index = None, history_size = uni_past_history,
                                        target_size = uni_future_target)
  #-------------------- DATA IN TENSORFLOW FORMAT ----------------------
  #Train data
  train_single_uni = tf.data.Dataset.from_tensor_slices((x_train_single_uni, y_train_single_uni))
  train_single_uni = train_single_uni.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
  #Validation data
  val_single_uni = tf.data.Dataset.from_tensor_slices((x_val_single_uni,y_val_single_uni))
  val_single_uni = val_single_uni.batch(BATCH_SIZE).repeat()

  for i in range(N_SIMS):
    print("Simulation:"+str(i+1))
    #-------------------- MODEL ----------------------
    tf.keras.backend.clear_session()
    single_step_1D_model = my_model(n_of_neurons=256,in_shape=x_train_single_uni.shape[-2:])
    single_step_1D_model.compile(optimizer='adam', loss='mae')

    #Training
    P = 3
    early_stopping = EarlyStopping(monitor='val_loss', patience=P, restore_best_weights=True)
    single_step_1D_model_history = single_step_1D_model.fit(train_single_uni,
                                                          epochs=EPOCHS,
                                                          steps_per_epoch=EVALUATION_INTERVAL,
                                                          validation_data=val_single_uni,
                                                          callbacks=[early_stopping],
                                                          validation_steps=EVALUATION_INTERVAL)
    #saving predictions (256 neurons)
    predictions256U = single_step_1D_model.predict(x_val_single_uni)
      
    #Calculating and saving errors
    MSE = round(mean_squared_error(y_val_single_uni, predictions256U , squared=True),3)
    errors_sim.append(MSE)

    #deleting model
    del single_step_1D_model, single_step_1D_model_history

  #print(errors_sim)
  df_errors.insert(loc=j, column=df_adj_lot.columns[j],value=errors_sim)

print(df_errors)
filename = "df_errors.csv"
df_errors.to_csv(filename, index=False)

Streaming output truncated to the last 5000 lines.
200/200 [==============================] - 3s 11ms/step - loss: 0.1371 - val_loss: 1.1717
Epoch 2/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0921 - val_loss: 1.0245
Epoch 3/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0859 - val_loss: 0.9826
Epoch 4/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0847 - val_loss: 0.9608
Epoch 5/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0831 - val_loss: 1.1474
Epoch 6/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0817 - val_loss: 1.1845
Epoch 7/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0795 - val_loss: 1.3162
Simulation:10
Epoch 1/100
200/200 [==============================] - 3s 10ms/step - loss: 0.1325 - val_loss: 1.2595
Epoch 2/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0928 - val_loss: 0.9891
Epoch 3/100
200/200 [===============

# Guardando los errores en un archivo

In [ ]:
#filename = "df_errors.csv"
#df_errors.to_csv(filename, index=False)
! ls

df_errors.csv  sample_data


In [ ]:
df_errs_file = pd.read_csv(filename)
print(df_errs_file)
print("---------- Bootstrap ----------")
random.seed(10)
bs_reps = dcst.draw_bs_reps(df_errs_file["AAPL"], np.mean, size = 10000)
conf_int = np.percentile(bs_reps, [2.5, 97.5])
#print(np.mean(errors))
#print(np.median(errors256))
print(conf_int)
#print(conf_int-np.mean(errors256))
#print(conf_int[1]-conf_int[0])

    AAPL    AMC  AUDUSD=X   BGCP  BTC-USD  ...  ^IXIC   ^NYA   ^RUT   ^VIX   ^XAX
0  0.067  0.213     0.012  0.010    1.392  ...  0.030  0.013  0.034  0.056  0.031
1  0.066  0.226     0.012  0.010    2.116  ...  0.059  0.013  0.031  0.054  0.025
2  0.095  0.215     0.012  0.010    1.696  ...  0.059  0.013  0.027  0.060  0.032
3  0.095  0.215     0.012  0.010    0.992  ...  0.027  0.013  0.034  0.059  0.036
4  0.079  0.220     0.013  0.009    1.095  ...  0.018  0.013  0.023  0.056  0.035
5  0.089  0.211     0.012  0.010    1.510  ...  0.040  0.014  0.030  0.057  0.029
6  0.109  0.215     0.012  0.010    1.079  ...  0.031  0.015  0.040  0.057  0.033
7  0.059  0.209     0.013  0.009    1.835  ...  0.018  0.015  0.026  0.055  0.030
8  0.102  0.222     0.012  0.009    1.193  ...  0.027  0.013  0.030  0.060  0.033
9  0.083  0.216     0.012  0.009    2.225  ...  0.028  0.014  0.024  0.058  0.028

[10 rows x 41 columns]
---------- Bootstrap ----------
[0.0745 0.094 ]


<a name="Conclu"></a>
#Conclusiones
1. ....

Volver a [Tabla de contenidos](#contents).



## Bibliografía
1. [Python Tutorial: Correlation of Two Time Series (2m 31s) - Coeficiente de Pearson para 2 series de tiempo](https://www.youtube.com/watch?v=XVV6IVPUlKU)